In [1]:
import gisttools as gt
import numpy as np

# Get $E_{bulk}^{ww}$ at 298.15 K and 328.15 K

In [2]:
T_init = 298.15
T_final = 328.15
dTemp = T_final - T_init

In [3]:
water_25 = gt.gist.load_gist_file(
    "04-pure-water-box/GIST-analysis/25deg/gist-output.dat"
)
water_25.eww_ref = water_25["Eww_norm"].mean()

/Users/jsetiadi/Documents/Software/gisttools/gisttools/gist.py:264: RuntimeWarning: eww reference is zero, but there is a Eww_unref_norm column. All operations that rely on referenced Eww values will generate wrong results. Explicitly set eww_ref to zero to suppress this warning.
  warnings.warn(RuntimeWarning(


In [4]:
water_55 = gt.gist.load_gist_file(
    "04-pure-water-box/GIST-analysis/55deg/gist-output.dat"
)
water_55.eww_ref = water_55["Eww_norm"].mean()

/Users/jsetiadi/Documents/Software/gisttools/gisttools/gist.py:264: RuntimeWarning: eww reference is zero, but there is a Eww_unref_norm column. All operations that rely on referenced Eww values will generate wrong results. Explicitly set eww_ref to zero to suppress this warning.
  warnings.warn(RuntimeWarning(


# Load `gist-output.dat` from `Complex`, `Host`, and `Guest`

In [5]:
complex_25 = gt.gist.load_gist_file(
    "01-complex-in-water/GIST-analysis/25deg/gist-output.dat", eww_ref=water_25.eww_ref
)
complex_55 = gt.gist.load_gist_file(
    "01-complex-in-water/GIST-analysis/55deg/gist-output.dat", eww_ref=water_55.eww_ref
)

In [6]:
host_25 = gt.gist.load_gist_file(
    "02-host-in-water/GIST-analysis/25deg/gist-output.dat", eww_ref=water_25.eww_ref
)
host_55 = gt.gist.load_gist_file(
    "02-host-in-water/GIST-analysis/55deg/gist-output.dat", eww_ref=water_55.eww_ref
)

In [7]:
guest_25 = gt.gist.load_gist_file(
    "03-guest-in-water/GIST-analysis/25deg/gist-output.dat", eww_ref=water_25.eww_ref
)
guest_55 = gt.gist.load_gist_file(
    "03-guest-in-water/GIST-analysis/55deg/gist-output.dat", eww_ref=water_55.eww_ref
)

# Get Total Energy -- $E_{i}^{tot} = E_{i}^{sw} + \left(E_{i}^{ww} - \frac{n_{i}}{n_{i,bulk}}E_{i,bulk}^{ww}\right) \times V_{voxel}$

In [8]:
complex_25["Etotal"] = (complex_25["Esw_dens"] + complex_25["Eww_dens"]) * np.prod(
    complex_25.grid.delta
)
complex_55["Etotal"] = (complex_55["Esw_dens"] + complex_55["Eww_dens"]) * np.prod(
    complex_55.grid.delta
)

In [9]:
host_25["Etotal"] = (host_25["Esw_dens"] + host_25["Eww_dens"]) * np.prod(
    host_25.grid.delta
)
host_55["Etotal"] = (host_55["Esw_dens"] + host_55["Eww_dens"]) * np.prod(
    host_55.grid.delta
)

In [10]:
guest_25["Etotal"] = (guest_25["Esw_dens"] + guest_25["Eww_dens"]) * np.prod(
    guest_25.grid.delta
)
guest_55["Etotal"] = (guest_55["Esw_dens"] + guest_55["Eww_dens"]) * np.prod(
    guest_55.grid.delta
)

# Get Heat Capacity -- $C_{p,i} = \frac{E_{i}^{tot}(T_{2}) - E_{i}^{tot}(T_{1})}{T_{2} - T_{1}} \times 1000$ cal/mol/K

In [11]:
Cp_complex = (complex_55["Etotal"] - complex_25["Etotal"]) / dTemp * 1000
Cp_host = (host_55["Etotal"] - host_25["Etotal"]) / dTemp * 1000
Cp_guest = (guest_55["Etotal"] - guest_25["Etotal"]) / dTemp * 1000

# Change in Heat Capacity Upon Binding -- $\Delta C_{p,b} = C_{p}^{complex} - C_{p}^{host} - C_{p}^{guest}$

In [12]:
dCp = Cp_complex - Cp_host - Cp_guest

print(f"Delta Cp(b) = {dCp.sum():.1f} cal/mol/K")

Delta Cp(b) = -115.4 cal/mol/K


# Save Heat Capacity to DX file

In [13]:
complex_25["Heat_Capacity"] = dCp
complex_25.save_dx("Heat_Capacity", "heat_capacity_binding.dx")